In [7]:
import glob
import re
import pandas as pd
from pandas import DataFrame
from yattag import Doc
import unicodedata

In [9]:
doc, tag, text = Doc().tagtext()
txt_file = glob.glob('AmostrasDODF/**/*.txt')

all_contracts = []

for txt in txt_file:
    instance = re.findall(r'[\s\S]+', 
                unicodedata.normalize('NFKD', open(txt, encoding='utf-8').read()))

    instance = instance[0].split('xxbcet SEÇÃO III xxecet')
    all_contracts.append(txt + '\n' + instance[-1])


secao_3 = []

for contract in all_contracts:
    secao_3.append(contract.split('\n'))


In [16]:
len(secao_3)

555

### DFA

- l = lista com toda a seção 3 de um único dodf 
- i = posição atual da lista l -> inicia-se com 0
- l_acts = lista vazia que será preenchida com os atos
- j = posição atual da lista l_acts -> inicia-se com 0
- regex = regex do ato que está sendo procurado

In [11]:
def q0(l, i, l_acts, j):
    """Estado que procura por um ato de licitação"""

    regex5 = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+RESULTADO\s+D[EO]\s+JULGAMENTO|AVISO\s+D[EO]\s+JULGAMENTO)'
    regex6 = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+LICITAÇÃO)'
    regex7 = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+JULGAMENTO\s+D[EO]\s+HABILITAÇÃO)'
    regex8 = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+RESULTADO\s+D[EO]\s+RECURSO\s+E\s+JULGAMENTO)'

    # regex5 = r'(?:xxbcet\s+)?(?:AVISO\s+DE\s+ADJUDICAC[CÇ][AÃ]O\s+E\s+HOMOLOGA[CÇ][AÃ]O|AVISO\s+DE\s+HOMOLOGA[CÇ][AÃ]O\s+E\s+ADJUDICA[CÇ][AÃ]O)'# check
    # regex6 = r'(?:xxbcet\s+)?(?:AVISO\s+DE\s+HOMOLOGA[CÇ][AÃ]O\s+E\s+CONVOCA[CÇ][AÃ]O|AVISO\s+DE\s+CONVOCA[CÇ][AÃ]O)'# check
    # regex7 = r'(?:xxbcet\s+)?(?:AVISO\s+DE\s+HOMOLOGA[CÇ][AÃ]O)'# check
    # regex8 = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+DECLARACAO\s+D[EO]\s+VENCEDOR|AVISO\s+D[EO]\s+RESULTADO)'# check
    # regex9 = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITACAO|AVISO\s+D[EO]\s+ABERTURA)'# check

    if i == len(l):
        return


    elif re.match(regex5, l[i]):
        l_acts.append('[5] ' + l[0] + l[i])
        q1(l, i + 1, l_acts, j, regex5)

    elif re.match(regex6, l[i]):
        l_acts.append('[6] ' + l[0] + l[i])
        q1(l, i + 1, l_acts, j, regex6)

    elif re.match(regex7, l[i]):
        l_acts.append('[7] ' + l[0] + l[i])
        q1(l, i + 1, l_acts, j, regex7)

    elif re.match(regex8, l[i]):
        l_acts.append('[8] ' + l[0] + l[i])
        q1(l, i + 1, l_acts, j, regex8)

    # elif re.match(regex9, l[i]):
    #     l_acts.append('[9] ' + l[0] + l[i])
    #     q1(l, i + 1, l_acts, j, regex9)

    else:
        q0(l, i + 1, l_acts, j)


def q1(l, i, l_acts, j, regex):
    """Estado que procura a continuação do ato de licitação"""

    if i == len(l)-1:
        return

    regex_s = r'xxbcet (?:AVISOS?|EXTRATOS?|RESULTADOS?|SECRETARIA|PREGÃO|TOMADA|COMISSÃO)'

    if not re.match(regex, l[i]) and re.match(regex_s, l[i]) and ('xxbob' in l[i-1] or('—' in l[i-1] and 'xxbob' in l[i-2])):
        q0(l, i, l_acts, j + 1)

    elif re.match(regex, l[i]):
        q0(l, i, l_acts, j + 1)

    else:
        l_acts[j] += '\n' + l[i]
        q1(l, i + 1, l_acts, j, regex)



In [12]:
acts = []

for idx in secao_3:
    l_acts = []
    q0(idx, 0, l_acts, 0)
    acts.extend(l_acts)


In [13]:
len(acts)

87

In [14]:
acts[0]

'[6] AmostrasDODF\\DODF 008 11-01-2012 SECAO3_txt\\004.txtxxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃO ELETRÔNICO No 8/2012. xxecet\nACOMPANHIA DE SANEAMENTO AMBIENTAL DO DISTRITO FEDERAL torna público\nque realizará o Pregão Eletrônico PE-008/2012-CAESB, processo no 092.011281/2011, Aquisi-\nção de Reagentes Químicos e Vidrarias, conforme especificações técnicas constantes no Anexo\nI(Pedido de Aquisição/Termo de Referência), por preço equalizado, por lote cotado. Data final\nxxeob\n\nxxbob\npara Recebimento das Propostas: 26 de janeiro de 2012, às 08h30. Início da Sessão de Disputa:\n26 de janeiro 2012, às 10h00. O edital e seus anexos poderão ser encontrados no site www.\nlicitacoes-e.com.br a partir do dia 12 de janeiro de 2012. Outras informações poderão ser obtidas\npor meio do telefone (61) 3213-7275, email pregao(dcaesb.df.gov.br.\nxxbcet Brasília/DF, 10 de janeiro de 2012. xxecet\nxxbcet RAULINDO JUNIOR NAVES REZENDE xxecet\nxxbcet Pregoeiro Caesb

In [15]:
acts

['[6] AmostrasDODF\\DODF 008 11-01-2012 SECAO3_txt\\004.txtxxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃO ELETRÔNICO No 8/2012. xxecet\nACOMPANHIA DE SANEAMENTO AMBIENTAL DO DISTRITO FEDERAL torna público\nque realizará o Pregão Eletrônico PE-008/2012-CAESB, processo no 092.011281/2011, Aquisi-\nção de Reagentes Químicos e Vidrarias, conforme especificações técnicas constantes no Anexo\nI(Pedido de Aquisição/Termo de Referência), por preço equalizado, por lote cotado. Data final\nxxeob\n\nxxbob\npara Recebimento das Propostas: 26 de janeiro de 2012, às 08h30. Início da Sessão de Disputa:\n26 de janeiro 2012, às 10h00. O edital e seus anexos poderão ser encontrados no site www.\nlicitacoes-e.com.br a partir do dia 12 de janeiro de 2012. Outras informações poderão ser obtidas\npor meio do telefone (61) 3213-7275, email pregao(dcaesb.df.gov.br.\nxxbcet Brasília/DF, 10 de janeiro de 2012. xxecet\nxxbcet RAULINDO JUNIOR NAVES REZENDE xxecet\nxxbcet Pregoeiro Caes

In [16]:
df = DataFrame(acts,columns=['Texto'])

In [17]:
df

,Texto
0,[6] AmostrasDODF\DODF 008 11-01-2012 SECAO3_tx...
1,[5] AmostrasDODF\DODF 008 11-01-2012 SECAO3_tx...
2,[5] AmostrasDODF\DODF 008 11-01-2012 SECAO3_tx...
3,[6] AmostrasDODF\DODF 008 11-01-2012 SECAO3_tx...
4,[6] AmostrasDODF\DODF 008 11-01-2012 SECAO3_tx...
...,...
82,[6] AmostrasDODF\DODF 209 30-10-2007 SECAO3_tx...
83,[6] AmostrasDODF\DODF 209 30-10-2007 SECAO3_tx...
84,[6] AmostrasDODF\DODF 209 30-10-2007 SECAO3_tx...
85,[5] AmostrasDODF\DODF 229 06-12-2005 SECAO3_tx...


In [18]:
df = DataFrame(acts,columns=['Texto'])

# Regex para encontrar o id do ato
df['Id_ato'] =  df['Texto'].apply(lambda st: st[1:2])
re_exp = r'\[5\]|\[6\]|\[7\]|\[8\]'
df['Texto'] =  df['Texto'].str.replace(re_exp,'')


# Regex para encontrar o dodf de onde está sendo retirado a informação
df['Dodf'] =  df['Texto'].apply(lambda st: st.split('.txt')[0])
df['Texto'] =  df['Texto'].apply(lambda st: st.split('.txt')[1])


# Regex para encontrar o processo do ato
df['Processo'] = df['Texto'].str.extract(r'''(?:(?:(?:P|p)rocesso(?:\s+)?(?:(?:\()?SEI(?:\)?))?(?:\s+)?(?:(?:no|n\.o)?)?)|(?:P|p)rocesso:|(?:P|p)rocesso|Processo.|(?:P|p)rocesso\s+no|(?:P|p)rocesso\s+n.? ?o.?|(?:P|p)rocesso\s+no:|(?:P|p)rocesso\s+SEI\s+no:|(?:P|p)rocesso\s+SEI:|(?:P|p)rocesso\s+SEI-GDF:|(?:P|p)rocesso\s+SEI-GDF|(?:P|p)rocesso\s+SEI\s+no|(?:P|p)rocesso\s+SEI\s+n|(?:P|p)rocesso\s+SEI|(?:P|p)rocesso-\s+SEI|(?:P|p)rocesso\s+SEI\s+no.|(?:P|p)rocesso\s+\(SEI\)\s+no.|(?:P|p)rocesso\s+SEI\.|(?:P|p)rocesso\s+\(SEI-DF\)\s+no.|(?:P|p)rocesso\s+SEI-GDF no|(?:P|p)rocesso\s+n|(?:P|p)rocesso\s+N|(?:P|p)rocesso\s+administrativo no|(?:P|p)rocesso\s+n:|PROCESSO: ?N?o?|PROCES-? ?SO|PROCESSO.|PROCESSO\s+no|PROCESSO\s+No|PROCESSO\s+N.o:?|PROCESSO\s+no.|PROCESSO\s+no:|PROCESSO\s+No:|PROCESSO\s+SEI\s+no:|PROCESSO\s+SEI:|PROCESSO\s+SEI|PROCESSO\s+SEI-GDF:|PROCESSO\s+SEI-GDF|PROCESSO\s+SEI\s+no|PROCESSO\s+SEI\s+No|PROCESSO\s+SEI\s+no.|PROCESSO\s+SEI.)((?:(?!\s\d{2}.\d{3}.\d{3}/\d{4}-\d{2}))(?:(?:\s)(?:(?:[\d.]+)|(?:[\d\s]+))[.-]?(?:(?:\d)|(?:[.\d\sSEI-|]+))(?:/|-
\b)(?:(?:(?:\d)+|(?:[\d\s]+)))?(?:-(?:(?:\d)+|(?:[\d\s]+)))?(?:-SECOM/DF|-?CEB|/CBMDF|F J Z B / D F)?))''')


# Regex para encontrar a data do ato
df['Data_dodf'] = df['Texto'].str.extract(r'''Brasí?i?lia(?:\/?DF)?,?\s+(\d{2}\s+de+\s\w+\s+de\s+\d{4})''')
for row in range(len(df)):
    if pd.isna(df['Data_dodf'][row]):
        df['Data_dodf'][row] = re.findall(r'\d{2}-\d{2}-\d{4}', df['Dodf'][row])[0]




In [19]:
df

,Texto,Id_ato,Dodf,Processo,Data_dodf
0,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PRE...,6,AmostrasDODF\DODF 008 11-01-2012 SECAO3_txt\004,092.011281/2011,10 de janeiro de 2012
1,xxbcet AVISO DO RESULTADO DE JULGAMENTO xxecet...,5,AmostrasDODF\DODF 008 11-01-2012 SECAO3_txt\015,060.004.060/2011,10 de janeiro de 2012
2,xxbcet AVISO DO RESULTADO DE JULGAMENTO DESERT...,5,AmostrasDODF\DODF 008 11-01-2012 SECAO3_txt\015,\n411.000.335/2010,10 de janeiro de 2012
3,xxbcet AVISO DE LICITAÇÃO DESERTA xxecet\nxx...,6,AmostrasDODF\DODF 008 11-01-2012 SECAO3_txt\016,411.000.075/2011,10 de janeiro de 2011
4,xxbcet AVISO DE LICITAÇÃO NOVA DATA xxecet\n...,6,AmostrasDODF\DODF 008 11-01-2012 SECAO3_txt\016,NaN,10 de janeiro de 2012
...,...,...,...,...,...
82,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet TOM...,6,AmostrasDODF\DODF 209 30-10-2007 SECAO3_txt\001,137.000.385/\n2007,26 de outubro de 2007
83,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet CON...,6,AmostrasDODF\DODF 209 30-10-2007 SECAO3_txt\008,NaN,29 de outubro de 2007
84,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet CON...,6,AmostrasDODF\DODF 209 30-10-2007 SECAO3_txt\008,00092.004.823/2007,29 de outubro de 2007
85,xxbcet AVISO DE RESULTADO DE JULGAMENTO xxecet...,5,AmostrasDODF\DODF 229 06-12-2005 SECAO3_txt\002,052.000.922/2004,05 de dezembro de 2005


In [20]:
df.isnull().sum()

Texto         0
Id_ato        0
Dodf          0
Processo     24
Data_dodf     0
dtype: int64

In [22]:
df['Texto'][72]

'xxbcet AVISO DE JULGAMENTO DE LICITAÇÃO xxecet\nxxbcet TOMADA DE PREÇOS No 07/2010. xxecet\nO PRESIDENTE DA COMISSÃO PERMANENTE DE LICITAÇÕES, DA COMPANHIA\nDE SANEAMENTO AMBIENTAL DO DISTRITO FEDERAL, TORNA PÚBLICO o\nresultado do julgamento da Tomada de Preços no 07/2010-CAESB, processo 092.001.810/\n2010, tipo de licitação: menor preço, para contratação de empresa especializada para serviços\nde projeto, fabricação e fornecimento de 01 (um) Centro de Controle de Motores (CCM) para\na Elevatória Booster do Lago Sul, EBO.LS.02, do Sistema de Abastecimento de Água da Caesb,\nna forma de execução indireta, sob regime de empreitada por preço global, da forma que se\nsegue: a empresa AG TECH SISTEMAS LTDA - EPP julgada vencedora com o valor global de\nR$ 80.000,00 (oitenta mil reais).\nxxbcet Brasília/DF, 27 de agosto de 2010 xxecet\nxxbcet JERÔNIMO BAREICHA JÚNIOR xxecet\nxxeob\n\nxxbob'

## Texto com as tags de marcação

In [32]:
# data = df[['Texto']]
# data

In [ ]:
# data.to_csv('CSVs/atos_with_tags.csv')

## Texto sem as tags de marcação

In [33]:
# re_exp = r'xxbcet ?|xxbcet ?|xxeob ?|xxbob ?|xxecet ?'
# data['Texto'] =  data['Texto'].str.replace(re_exp,'')
# data

In [31]:
# data.to_csv('CSVs/atos_without_tags.csv')